In [1]:
# Importing dependencies
import pandas as pd
import string
import re
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp_spacy = spacy.load("en_core_web_sm")

In [2]:
# Reading combined data 
comb_data=pd.read_csv(r"Combined_Categories_dataset.csv")
comb_data.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [3]:
reviews=comb_data[['reviewText']]
reviews.head(10)

,reviewText
0,Exactly what I needed.
1,"I agree with the other review, the opening is ..."
2,Love these... I am going to order another pack...
3,too tiny an opening
4,Okay
5,Exactly what I wanted.
6,These little plastic backs work great. No mor...
7,mother - in - law wanted it as a present for h...
8,"Item is of good quality. Looks great, too. But..."
9,I had used my last el-cheapo fake leather ciga...


# Take the first 10  review texts each from both the categories (20 samples in total).

In [4]:
# Getting 10 reviews from each category
reviews_10_amazon = reviews.iloc[0:10]
reviews_10_electronics = reviews.iloc[20000:20010]
reviews_20 = pd.concat([reviews_10_amazon, reviews_10_electronics], ignore_index=True)


In [5]:
reviews_20.shape

(20, 1)

In [6]:
reviews_20

,reviewText
0,Exactly what I needed.
1,"I agree with the other review, the opening is ..."
2,Love these... I am going to order another pack...
3,too tiny an opening
4,Okay
5,Exactly what I wanted.
6,These little plastic backs work great. No mor...
7,mother - in - law wanted it as a present for h...
8,"Item is of good quality. Looks great, too. But..."
9,I had used my last el-cheapo fake leather ciga...


In [8]:
# Breaking review in sentences
reviews_20['Review_sentences']=''
reviews_20['review_ID']=''
for i in range(reviews_20.shape[0]):
    reviews_20['review_ID'].iloc[i]=i
    text=nlp_spacy(reviews_20['reviewText'].iloc[i]) 
    reviews_20['Review_sentences'].iloc[i]=[]
    for sent in text.sents:
        reviews_20['Review_sentences'].iloc[i].append(sent.text)
        
output=reviews_20[['review_ID','Review_sentences']]
print(output)     

   review_ID                                   Review_sentences
0          0                           [Exactly what I needed.]
1          1  [I agree with the other review, the opening is...
2          2  [Love these..., I am going to order another pa...
3          3                              [too tiny an opening]
4          4                                             [Okay]
5          5                           [Exactly what I wanted.]
6          6  [These little plastic backs work great.  , No ...
7          7  [mother - in - law wanted it as a present for ...
8          8  [Item is of good quality., Looks great, too., ...
9          9  [I had used my last el-cheapo fake leather cig...
10        10  [This was the first time I read Garcia-Aguiler...
11        11  [As with all of Ms. Garcia-Aguilera's books, I...
12        12  [I've not read any of Ms Aguilera's works befo...
13        13  [This romance novel is right up there with the...
14        14  [Carolina Garcia Aguilera 

In [9]:
# Spliting each sentence in seperate rows
x=output.explode('Review_sentences')
x

,review_ID,Review_sentences
0,0,Exactly what I needed.
1,1,"I agree with the other review, the opening is ..."
1,1,I almost bent the hook on some very expensive ...
1,1,"Would not buy again but for the price, not sen..."
2,2,Love these...
...,...,...
19,19,"He becomes an infamous, opportunistic, connivi..."
19,19,Files keeps the novel fresh and hip.
19,19,"Through Penn, the reader hangs with the ""in cr..."
19,19,If readers are adventurous enough to veer away...
